conda install openpyxl

In [1]:
import pandas as pd  
from datetime import datetime, timedelta
import re
import os
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words

In [31]:
#กำหนด input file / ชื่อ output file
input='RawDataCSV\Biore UV Aqua Rich Watery Essence_20240924_192601.csv'
output='Biore UV Aqua Rich Watery Essence_20240924_192601.csv'

<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
C:\Users\earns\AppData\Local\Temp\ipykernel_7948\2303733208.py:2: SyntaxWarning: invalid escape sequence '\B'
  input='RawDataCSV\Biore UV Aqua Rich Watery Essence_20240924_192601.csv'


In [32]:
def convert_date(date_str, file_date):
    # แปลงวันที่จากชื่อไฟล์
    file_date = datetime.strptime(file_date, '%Y%m%d')
    
    # กรณีที่มีคำว่า 'ก่อน'
    if 'ก่อน' in date_str:
        value, unit = date_str.split()[:2]
        value = int(value)
        
        if unit in ['วินาทีก่อน', 'นาทีก่อน', 'ชั่วโมงก่อน']:
            return file_date.strftime('%Y%m%d')
        elif unit == 'วันก่อน':
            return (file_date - timedelta(days=value)).strftime('%Y-%m-%d')
        elif unit == 'สัปดาห์ก่อน':
            return (file_date - timedelta(weeks=value)).strftime('%Y-%m-%d')
    
    month_dict = {
        'ม.ค.': '01', 'ก.พ.': '02', 'มี.ค.': '03', 'เม.ย.': '04',
        'พ.ค.': '05', 'มิ.ย.': '06', 'ก.ค.': '07', 'ส.ค.': '08',
        'ก.ย.': '09', 'ต.ค.': '10', 'พ.ย.': '11', 'ธ.ค.': '12'
    }
    
    parts = date_str.split()
    if len(parts) == 3 and parts[1] in month_dict:
        day, month, year = parts
        return f"{year}-{month_dict[month]}-{int(day):02d}"
    
    return date_str

def process_dataframe(file_path):
    # อ่านไฟล์ CSV
    df = pd.read_csv(file_path)
    
    # แยกวันที่จากชื่อไฟล์
    file_date = file_path.split('_')[-2]
    
    # แปลงคอลัมน์ date
    df['date'] = df['date'].apply(lambda x: convert_date(x, file_date))
    
    return df


In [33]:
df = process_dataframe(input)
print(df)

        no                        shop             name  rating        date  \
0        1  Kao Beauty & Personal Care            น***.       5  2024-08-12   
1        2  Kao Beauty & Personal Care        กรานต์ ไ.       5  2024-07-07   
2        3  Kao Beauty & Personal Care           poonim       5  2024-06-25   
3        4  Kao Beauty & Personal Care            น***.       5  2024-06-29   
4        5  Kao Beauty & Personal Care            K***.       5  2024-06-22   
...    ...                         ...              ...     ...         ...   
1120  1121                       Boots            อ***.       5  2023-10-19   
1121  1122                       Boots  Pattarawijittra       5  2024-03-15   
1122  1123                       Boots       *******019       5  2023-12-04   
1123  1124                       Boots          Jirawan       5  2023-08-21   
1124  1125                       Boots        สุบงกช เ.       5  2023-04-28   

                                            review_

In [34]:
df.dtypes

no              int64
shop           object
name           object
rating          int64
date           object
review_text    object
dtype: object

In [35]:
# ฟังก์ชันลบข้อมูลที่ไม่ใช่ภาษาไทย
def clean_review(text):
    # ลบอีโมจิ, ภาษาอังกฤษ, ตัวเลข, สัญลักษณ์พิเศษ และช่องว่างเกินจำเป็น
    cleaned_text = re.sub(r'[^\u0E00-\u0E7F\s]', '', text)  # \u0E00-\u0E7F คือตัวอักษรภาษาไทยใน Unicode
    return cleaned_text.strip()

# ใช้ฟังก์ชัน clean_review กับคอลัมน์ review_text
df['review_text'] = df['review_text'].apply(lambda x: clean_review(str(x)))

In [36]:
# ลบการขึ้นบรรทัดใหม่ใน review_text และทำให้ข้อความอยู่ในบรรทัดเดียวกัน
df['review_text'] = df['review_text'].apply(lambda x: str(x).replace('\n', ' ').strip())

In [37]:
# โหลดฐานข้อมูลคำพื้นฐาน
word_list = set(thai_words())

def is_meaningful(word):
    # เพิ่มการตรวจสอบคำที่มีรูปแบบพิเศษ
    special_patterns = [
        r'^[ก-ฮ]+$',  # คำภาษาไทยทั่วไป
        r'^[ก-ฮ]+ๆ$',  # คำซ้ำ เช่น ดีๆ เร็วๆ
        r'^[ก-ฮ]+[ุึ][ปบ]$',  # คำลงท้ายด้วย ุป, ึป, ุบ, ึบ เช่น ปุ๊ป, ปั๊บ
        r'^แพ[็ค]$',  # คำว่า แพ็ค หรือ แพค
        r'^[ก-ฮ]{1,2}่[ก-ฮ]$',  # คำสั้นๆ ที่มีวรรณยุกต์ เช่น ค่ะ ไว
        r'^[0-9๐-๙]+$',  # ตัวเลข
        r'^[a-zA-Z]+$',  # คำภาษาอังกฤษ
    ]
    
    return (word in word_list or
            any(re.match(pattern, word) for pattern in special_patterns))

def clean_text(text):
    # แยกคำ
    words = word_tokenize(text)
    
    # แทนที่คำที่ไม่มีความหมายด้วยช่องว่าง
    cleaned_words = [word if is_meaningful(word) else ' ' for word in words]
    
    # รวมคำกลับเป็นประโยค
    cleaned_text = ''.join(cleaned_words)
    
    # ลบช่องว่างซ้ำ
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # ลบช่องว่างที่ไม่จำเป็นตอนต้นและท้ายข้อความ
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

In [38]:
df['review_text'] = df['review_text'].apply(clean_text)

In [39]:
# ลบแถวที่ review_text ว่าง (ที่ไม่มีข้อมูลภาษาไทยหรือเป็นช่องว่าง)
df_cleaned = df[df['review_text'].astype(bool)].copy()

In [40]:
df_cleaned

,no,shop,name,rating,date,review_text
0,1,Kao Beauty & Personal Care,น***.,5,2024-08-12,สูตรไม่เหนียวเหนอะหนะ ปกป้องผิวจากแสง และ กันเ...
1,2,Kao Beauty & Personal Care,กรานต์ ไ.,5,2024-07-07,ปกติซื่อแต่ในห้าง ราคาปกติไม่เคยไม่มีส่วนลด เป...
2,3,Kao Beauty & Personal Care,poonim,5,2024-06-25,อ่อนโยนต่อผิวที่แสบร้อน ไม่เกิดคราบขาวบนผิว เบ...
3,4,Kao Beauty & Personal Care,น***.,5,2024-06-29,ให้ผลลัพธ์ธรรมชาติ ใช้ง่ายและผสมเข้ากับผิวได้ด...
4,5,Kao Beauty & Personal Care,K***.,5,2024-06-22,คุณภาพ ดี เนื้อสัมผัส ซึมง่าย ประสิทธิภาพ ใช้ป...
...,...,...,...,...,...,...
1116,1117,Boots,Kck,4,2024-07-03,เบาสบายและไม่มัน สูตรเจลทำให้ใช้ง่าย
1117,1118,Boots,บุปผา,4,2022-06-30,ไม่เหนี่ยว สบายผิว
1118,1119,Boots,****1,4,2022-07-10,ดีค่ะ
1119,1120,Boots,จ***.,5,2024-09-10,ไม่อุดตันรูขุมขน สูตรกันน้ำเพื่อการปกป้องที่ยา...


In [41]:
# รีเซ็ตเลขลำดับคอลัมน์ 'no' ใน DataFrame ที่ผ่านการตรวจสอบ
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned['no'] = df_cleaned.index + 1

In [42]:
df_cleaned

,no,shop,name,rating,date,review_text
0,1,Kao Beauty & Personal Care,น***.,5,2024-08-12,สูตรไม่เหนียวเหนอะหนะ ปกป้องผิวจากแสง และ กันเ...
1,2,Kao Beauty & Personal Care,กรานต์ ไ.,5,2024-07-07,ปกติซื่อแต่ในห้าง ราคาปกติไม่เคยไม่มีส่วนลด เป...
2,3,Kao Beauty & Personal Care,poonim,5,2024-06-25,อ่อนโยนต่อผิวที่แสบร้อน ไม่เกิดคราบขาวบนผิว เบ...
3,4,Kao Beauty & Personal Care,น***.,5,2024-06-29,ให้ผลลัพธ์ธรรมชาติ ใช้ง่ายและผสมเข้ากับผิวได้ด...
4,5,Kao Beauty & Personal Care,K***.,5,2024-06-22,คุณภาพ ดี เนื้อสัมผัส ซึมง่าย ประสิทธิภาพ ใช้ป...
...,...,...,...,...,...,...
807,808,Boots,Kck,4,2024-07-03,เบาสบายและไม่มัน สูตรเจลทำให้ใช้ง่าย
808,809,Boots,บุปผา,4,2022-06-30,ไม่เหนี่ยว สบายผิว
809,810,Boots,****1,4,2022-07-10,ดีค่ะ
810,811,Boots,จ***.,5,2024-09-10,ไม่อุดตันรูขุมขน สูตรกันน้ำเพื่อการปกป้องที่ยา...


In [23]:
if not os.path.exists('CleanDataCSV'):
    os.makedirs('CleanDataCSV')
if not os.path.exists('CleanDataExel'):
    os.makedirs('CleanDataExel')

In [43]:
# บันทึก DataFrame ลงไฟล์ CSV
csv_file = f'CleanDataCSV/{output}.csv'
df_cleaned.to_csv(csv_file, index=False)

# บันทึก DataFrame ลงไฟล์ Excel
excel_file = f'CleanDataExel/{output}.xlsx'
df_cleaned.to_excel(excel_file, index=False)

In [1]:
import re
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp.spell import correct

# โหลดฐานข้อมูลคำพื้นฐาน
word_list = set(thai_words())

def is_meaningful(word):
    # เพิ่มการตรวจสอบคำที่มีรูปแบบพิเศษ
    special_patterns = [
        r'^[ก-ฮ]+$',  # คำภาษาไทยทั่วไป
        r'^[ก-ฮ]+ๆ$',  # คำซ้ำ เช่น ดีๆ เร็วๆ
        r'^[ก-ฮ]+[ุึ][ปบ]$',  # คำลงท้ายด้วย ุป, ึป, ุบ, ึบ เช่น ปุ๊ป, ปั๊บ
        r'^แพ[็ค]$',  # คำว่า แพ็ค หรือ แพค
        r'^[ก-ฮ]{1,2}่[ก-ฮ]$',  # คำสั้นๆ ที่มีวรรณยุกต์ เช่น ค่ะ ไว
        r'^[0-9๐-๙]+$',  # ตัวเลข
        r'^[a-zA-Z]+$',  # คำภาษาอังกฤษ
    ]
    
    return (word in word_list or
            any(re.match(pattern, word) for pattern in special_patterns))

def clean_text(text):
    # แยกคำ
    words = word_tokenize(text)
    
    # แทนที่คำที่ไม่มีความหมายด้วยช่องว่าง
    cleaned_words = [word if is_meaningful(word) else ' ' for word in words]
    
    # รวมคำกลับเป็นประโยค
    cleaned_text = ''.join(cleaned_words)
    
    # ลบช่องว่างซ้ำ
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # ลบช่องว่างที่ไม่จำเป็นตอนต้นและท้ายข้อความ
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

# ตัวอย่างการใช้งาน
examples = [
    "ใช้งานดีมากค่ะ กันแดดเนื้อเจล ปาดปุ๊ปซึมปั๊ปปปปปปป",
    "ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึมเร็ว ไม่เหนียวเหมาะสำหรับใช้ในชีวิตประจำวัน",
    "นใำนฝำนฝพนฝกนฝดสฝดนฝะัตงพนฝะนฝพนพนใำนใพนฝำนวำนฝดรใดรใด่มดรมำรมำรใพรใพนใพนฝพนฝพนฝพนฝดนใพน",
    "ดีมากๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆ",
    "ั  ชอบมาก ถูกมาก ดีมาก คุ้มค่า คุ้มราคาสุดๆ",
    "วแ่วแว่แว่อทใ ทใว่แ่วแีวแว่ งาอยีดัวแฝท  าฃอ่วแง่ งา ว่แีววีแวาแ งา ง่แางอางดีงแาง สฃอบรแยีแงาอางอบรเหมาะสำหรับกิจกรรมกลางแจ้ง, เหมาะสำหรับการใช้งานทุกวัน,"
]

for example in examples:
    print("Original:", example)
    print("Cleaned:", clean_text(example))
    print()

# สำหรับการใช้งานกับ DataFrame
# df['cleaned_review'] = df['review_text'].apply(clean_text)
# print(df[['review_text', 'cleaned_review']].head())

Original: ใช้งานดีมากค่ะ กันแดดเนื้อเจล ปาดปุ๊ปซึมปั๊ปปปปปปป
Cleaned: ใช้งานดีมากค่ะ กันแดดเนื้อเจล ปาดปุ๊ปซึมปั๊ปปปปปปป

Original: ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึมเร็ว ไม่เหนียวเหมาะสำหรับใช้ในชีวิตประจำวัน
Cleaned: ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึมเร็ว ไม่เหนียวเหมาะสำหรับใช้ในชีวิตประจำวัน

Original: นใำนฝำนฝพนฝกนฝดสฝดนฝะัตงพนฝะนฝพนพนใำนใพนฝำนวำนฝดรใดรใด่มดรมำรมำรใพรใพนใพนฝพนฝพนฝพนฝดนใพน
Cleaned: ใด

Original: ดีมากๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆ
Cleaned: ดีมาก

Original: ั  ชอบมาก ถูกมาก ดีมาก คุ้มค่า คุ้มราคาสุดๆ
Cleaned: ชอบมาก ถูกมาก ดีมาก คุ้มค่า คุ้มราคาสุด

Original: วแ่วแว่แว่อทใ ทใว่แ่วแีวแว่ งาอยีดัวแฝท  าฃอ่วแง่ งา ว่แีววีแวาแ งา ง่แางอางดีงแาง สฃอบรแยีแงาอางอบรเหมาะสำหรับกิจกรรมกลางแจ้ง, เหมาะสำหรับการใช้งานทุกวัน,
Cleaned: งาอยี แง่ งา วี งา อางดี แง อางอบรเหมาะสำหรับกิจกรรมกลางแจ้ง เหมาะสำหรับการใช้งานทุกวัน



In [93]:
from pythainlp.spell import correct

test = correct("ปุ๊ป")
print(test)

ปุ๊ป
